Importing Pakages and Data

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
df = pd.read_csv("CovidDeaths.csv")

In [ ]:
df1 = pd.read_csv('CovidVaccinations.csv')

In [ ]:
df1.head()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df1.info()

In [ ]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df1['date'] = pd.to_datetime(df1['date'], dayfirst=True)

In [ ]:
df.head()

Connecting to Sql

In [ ]:
cnn = sqlite3.connect('covid.db')

In [ ]:
df.to_sql('covid_deaths',cnn)

In [ ]:
df1.to_sql('covid_vaccinations',cnn)

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///covid.db

-- Birds eye view of the data sample

In [ ]:
%%sql

SELECT * 
FROM covid_deaths



In [ ]:
%%sql 

select * 
from covid_vaccinations


In [ ]:
%%sql 

select * 
from covid_vaccinations
order by 3,4


Select Data that we are going to be starting with

In [ ]:
%%sql

select Location,DATE(date) AS date_only,total_cases,new_cases,total_deaths,population
from covid_deaths
order by 1,2


Total Cases vs Total Deaths.
- Shows likelihood of dying if you contract covid


In [ ]:
%%sql

select Location,DATE(date) AS date_only,total_cases,total_deaths,population,(total_deaths/total_cases)*100 as death_percent
from covid_deaths
order by 1,2

In [ ]:
%%sql

select Location,DATE(date) AS date_only,total_cases,total_deaths,(total_deaths/total_cases)*100 as death_percent
from covid_deaths
where location = 'India'
order by 1,2

Total Cases vs Population.
- Shows the percentage of population infected with Covid

In [ ]:
%%sql

select Location,DATE(date) AS date_only,total_cases,population,(total_cases/population)*100 as case_percent
from covid_deaths
order by 1,2

In [ ]:
%%sql

select Location,DATE(date) AS date_only,total_cases,population,(total_cases/population)*100 as case_percent
from covid_deaths
where location = 'India'
order by 1,2

 Countries with Highest Infection Rate compared to Population

In [ ]:
%%sql

select Location,population,MAX(total_cases) as infection_count,MAX((total_cases/population))*100 as infection_percent
from covid_deaths
group by population, Location
order by infection_percent desc

Countries with Highest Death Count per Population

In [ ]:
%%sql

select Location,population,MAX(total_deaths) as death_count
from covid_deaths
where continent is not null
group by population, Location
order by death_count desc

Highest death count per population by Continents

In [ ]:
%%sql

select location,MAX(total_deaths) as death_count
from covid_deaths
where continent is null
group by location
order by death_count desc

In [ ]:
%%sql
select location, SUM(new_deaths) as TotalDeathCount
from covid_deaths
where continent is null 
and location not in ('World', 'European Union', 'International')
group by location
order by TotalDeathCount desc

Rolling number of new cases and new deaths by date 

In [ ]:
%%sql

select DATE(date) AS date_only, sum(new_cases), sum(new_deaths), sum(new_deaths)/sum(new_cases) *100 as death_percentage
from covid_deaths
group by date
order by 1,2


Global Numbers

In [ ]:
%%sql

select sum(new_cases) as total_cases, sum(new_deaths) as total_deaths, sum(new_deaths)/sum(new_cases) *100 as death_percentage
from covid_deaths
where continent is not null
order by 1,2

Total Population vs Vaccinations.

  ShowsNumber  of Population that has recieved at least one Covid Vaccine

In [ ]:
%%sql

select d.continent, d.location, d.date, d.population, MAX(v.total_vaccinations) as RollingPeopleVaccinated
from covid_deaths as d
join covid_vaccinations as v
	On d.location = v.location
	and d.date = v.date
where d.continent is not null 
group by d.continent, d.location, d.date, d.population
order by 1,2,3


In [ ]:
%%sql

select d.continent,d.location,d.date,d.population,v.new_vaccinations, sum(v.new_vaccinations) over(partition by d.location order by d.location,d.date) as RollingPeopleVaccinated
from covid_deaths as d
join covid_vaccinations as v
on d.location =v.location and d.date = v.date
where d.continent is not null
order by 2,3

    

Using CTE to perform Calculation on Partition By in previous query

In [ ]:
%%sql

with temp (continent,location,date,population,new_vaccinations,total_vac_till_date) 
as
(
select d.continent,d.location,d.date,d.population,v.new_vaccinations, 
    sum(v.new_vaccinations) over(partition by d.location order by d.location, d.date) as total_vac_till_date
from covid_deaths as d
join covid_vaccinations as v
on d.location =v.location and d.date = v.date
where d.continent is not null

)

select * , (total_vac_till_date/population)*100
from temp

Using Temp Table to perform Calculation on Partition By in previous query

In [ ]:
%%sql
create table percent_population_vac
(
    continent nvarchar(225),
    location nvarchar(225),
    date datetime,
    population numeric,
    new_vaccination numeric,
    total_vac_till_date numeric
)




In [ ]:
%%sql

insert into percent_population_vac

select d.continent,d.location,d.date,d.population,v.new_vaccinations, 
    sum(v.new_vaccinations) over(partition by d.location order by d.location, d.date) as total_vac_till_date
from covid_deaths as d
join covid_vaccinations as v
on d.location =v.location and d.date = v.date
where d.continent is not null




In [ ]:
%%sql

select * , (total_vac_till_date/population)*100
from percent_population_vac

Exporting some tables obtained from the above queries as csv file to perform visualizations in Tableau

In [ ]:
cursor = cnn.cursor()


In [ ]:

q1 = "SELECT sum(new_cases) as total_cases, sum(new_deaths) as total_deaths, sum(new_deaths)/sum(new_cases) *100 as death_percentage FROM covid_deaths ORDER BY 1,2"
cursor.execute(q1)

In [ ]:
death_percentage = pd.DataFrame(cursor.fetchall(), columns=[description[0] for description in cursor.description])

In [ ]:
death_percentage.to_csv('death_percent.csv', index=False) 

In [ ]:
q2 ="select location, SUM(new_deaths) as TotalDeathCount from covid_deaths where continent is null and location not in ('World', 'European Union', 'International') group by location order by TotalDeathCount desc"
cursor.execute(q2)
data2 = pd.DataFrame(cursor.fetchall(), columns=[description[0] for description in cursor.description])
data2.to_csv ('death_continent.csv', index = False)

In [ ]:
 q3 = " select Location,population,MAX(total_cases) as infection_count,MAX((total_cases/population))*100 as infection_percent from covid_deaths group by population, Location order by infection_percent desc"


In [ ]:
cursor.execute(q3)
data3 = pd.DataFrame(cursor.fetchall(), columns=[description[0] for description in cursor.description])


In [ ]:

data3 = data3.fillna(0)


In [ ]:
data3.to_csv ('infection_count.csv', index = False)

In [ ]:
%%sql

Select Location, population,DATE(date) AS date, MAX(total_cases) as HighestInfectionCount,  Max((total_cases/population))*100 as PercentPopulationInfected
From covid_deaths
Group by Location, Population, date
order by PercentPopulationInfected desc
limit 100

In [ ]:
q4 = "Select Location, population,DATE(date) AS date, MAX(total_cases) as HighestInfectionCount,  Max((total_cases/population))*100 as PercentPopulationInfected From covid_deaths Group by Location, Population, date order by PercentPopulationInfected desc"
cursor.execute(q4)
data4 = pd.DataFrame(cursor.fetchall(), columns=[description[0] for description in cursor.description])


In [ ]:

data4 = data4.fillna(0)

In [ ]:
data4.to_csv ('table4.csv', index = False)